In [5]:
'''
#testing
import torch
import time
def gputester(MATRIX_SIZE):
    # Define the size of the matrices (adjust for your system)

    # Create random matrices
    cpu_tensor1 = torch.randn(MATRIX_SIZE, MATRIX_SIZE)
    cpu_tensor2 = torch.randn(MATRIX_SIZE, MATRIX_SIZE)

    # Move tensors to GPU (if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gpu_tensor1 = cpu_tensor1.to(device)
    gpu_tensor2 = cpu_tensor2.to(device)

    ### **🔹 CPU Benchmark**
    start_cpu = time.time()
    cpu_result = torch.matmul(cpu_tensor1, cpu_tensor2)  # Matrix multiplication on CPU
    end_cpu = time.time()
    cpu_time = end_cpu - start_cpu
    print(f"✅ CPU Time: {cpu_time:.4f} seconds")

    ### **🔹 GPU Benchmark**
    # Ensure the GPU is warmed up
    torch.matmul(gpu_tensor1, gpu_tensor2)

    # Measure GPU performance
    torch.cuda.synchronize()  # Ensure operations are completed before timing
    start_gpu = time.time()
    gpu_result = torch.matmul(gpu_tensor1, gpu_tensor2)  # Matrix multiplication on GPU
    torch.cuda.synchronize()  # Ensure GPU computation is finished before stopping time
    end_gpu = time.time()
    gpu_time = end_gpu - start_gpu
    print(f"🚀 GPU Time: {gpu_time:.4f} seconds")

    ### **🔹 Speedup Factor**
    if torch.cuda.is_available():
        speedup = cpu_time / gpu_time
        print(f"🔥 GPU is {speedup:.2f}x faster than CPU!")
    else:
        print("⚠️ CUDA is not available! Running only on CPU.")

gputester(10000)
'''

'\n#testing\nimport torch\nimport time\ndef gputester(MATRIX_SIZE):\n    # Define the size of the matrices (adjust for your system)\n\n    # Create random matrices\n    cpu_tensor1 = torch.randn(MATRIX_SIZE, MATRIX_SIZE)\n    cpu_tensor2 = torch.randn(MATRIX_SIZE, MATRIX_SIZE)\n\n    # Move tensors to GPU (if available)\n    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n    gpu_tensor1 = cpu_tensor1.to(device)\n    gpu_tensor2 = cpu_tensor2.to(device)\n\n    ### **🔹 CPU Benchmark**\n    start_cpu = time.time()\n    cpu_result = torch.matmul(cpu_tensor1, cpu_tensor2)  # Matrix multiplication on CPU\n    end_cpu = time.time()\n    cpu_time = end_cpu - start_cpu\n    print(f"✅ CPU Time: {cpu_time:.4f} seconds")\n\n    ### **🔹 GPU Benchmark**\n    # Ensure the GPU is warmed up\n    torch.matmul(gpu_tensor1, gpu_tensor2)\n\n    # Measure GPU performance\n    torch.cuda.synchronize()  # Ensure operations are completed before timing\n    start_gpu = time.time()\n    g

In [96]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate

class RuleBasedPokerBot(BasePokerPlayer):
    
    def declare_action(self, valid_actions, hole_card, round_state):
        """ Selects Fold, Call, or Raise based on hand strength. """
        
        # Convert cards into the correct format
        hand_strength = estimate_hole_card_win_rate(
            nb_simulation=10, 
            nb_player=2,
            hole_card=gen_cards(hole_card),  # Convert to card objects
            community_card=gen_cards(round_state["community_card"])  # Convert community cards
        )

        action, amount = "fold", 0  

        if hand_strength >= 0:
            raise_action = next((a for a in valid_actions if a["action"] == "raise"), None)
            if raise_action:
                action, amount = "raise", raise_action["amount"]["min"]
            else:
                action, amount = "call", 0  # If raising is unavailable, call

        elif "call" in [a["action"] for a in valid_actions]:  
            action, amount = "call", 0  # Call doesn't need a bet amount
        print(f"RuleBasedPokerBot: {action, amount}")
        return action, amount  # Must return (action, amount)

    def receive_game_start_message(self, game_info):
        print(game_info)  
    def receive_round_start_message(self, round_count, hole_card, seats):
        print(f"RuleBasedPokerBot Hole Cards: {hole_card}\n")
    def receive_street_start_message(self, street, round_state): pass
    def receive_game_update_message(self, action, round_state): pass  
    def receive_round_result_message(self, winners, hand_info, round_state): pass



class AlwaysCallBot(BasePokerPlayer):
    def declare_action(self, valid_actions, hole_card, round_state):
        """ Always calls when possible, otherwise folds. """

        action, amount = "fold", 0  # Default to folding
        # Check if calling is an option
        if any(a["action"] == "call" for a in valid_actions):
            call_action_info = valid_actions[1]
            action, amount = "call", call_action_info["amount"]  # Call doesn't require a bet amount
        print(f"AlwaysCallBot: {action, amount}")
        return action, amount

    def receive_game_start_message(self, game_info): 
        print(game_info)   
    def receive_round_start_message(self, round_count, hole_card, seats):
        print(f"AlwaysCallBot Hole Cards: {hole_card}\n")              
    def receive_street_start_message(self, street, round_state): pass
    def receive_game_update_message(self, action, round_state): pass  
    def receive_round_result_message(self, winners, hand_info, round_state): pass


In [94]:
from pypokerengine.utils.game_state_utils import\
        restore_game_state, attach_hole_card, attach_hole_card_from_deck

def setup_game_state(round_state, my_hole_card):
    game_state = restore_game_state(round_state)
    for player_info in round_state['seats']:
        if uuid == self.uuid:
            # Hole card of my player should be fixed. Because we know it.
            game_state = attach_hole_card(game_state, uuid, my_hole_card)
        else:
            # We don't know hole card of opponents. So attach them at random from deck.
            game_state = attach_hole_card_from_deck(game_state, uuid)

In [216]:

from pypokerengine.api.emulator import Emulator
from pypokerengine.api.game import setup_config, start_poker


# Create emulator instance
emulator = Emulator()

# Set up the game rules

emulator.set_game_rule(player_num=2, max_round=2, small_blind_amount=5, ante_amount=0)

players_info = {
    "p1_uuid": {"name": "Player 1", "stack": 1000},
    "p2_uuid": {"name": "Player 2", "stack": 1000}
}

initial_game_state = emulator.generate_initial_game_state(players_info)

print(initial_game_state)
print("\n")
# Create player instances
p1_model = AlwaysCallBot()
p2_model = RuleBasedPokerBot()

# Register players (UUIDs are assigned automatically)
p1_uuid = "p1_uuid"
p2_uuid = "p2_uuid"

emulator.register_player(p1_uuid, p1_model)
emulator.register_player(p2_uuid, p2_model)


game_state, events = emulator.start_new_round(initial_game_state)
print(events[0]["round_state"])
print(events[0]["round_state"]["community_card"])
print(events[0]["round_state"]["seats"])
print("\n")


game_state, events = emulator.apply_action(game_state, "raise", 20)
print(events[0]["round_state"])
print(events[0]["round_state"]["community_card"])
print(events[0]["round_state"]["seats"])
print("\n")

game_state, events = emulator.apply_action(game_state, "call", 20)
print(events[0]["round_state"])
print(events[0]["round_state"]["community_card"])
print(events[0]["round_state"]["seats"])
print("\n")
game_state, events = emulator.apply_action(game_state, "call", 0)
print(events[0]["round_state"])
print(events[0]["round_state"]["community_card"])
print(events[0]["round_state"]["seats"])
print("\n")
game_state, events = emulator.apply_action(game_state, "call", 0)
print(events[0]["round_state"])
print(events[0]["round_state"]["community_card"])
print(events[0]["round_state"]["seats"])
print("\n")
game_state, events = emulator.apply_action(game_state, "call", 0)
print(events[0]["round_state"])
print(events[0]["round_state"]["community_card"])
print(events[0]["round_state"]["seats"])
print("\n")
game_state, events = emulator.apply_action(game_state, "call", 0)
print(events[0]["round_state"])
print(events[0]["round_state"]["community_card"])
print(events[0]["round_state"]["seats"])
print("\n")
game_state, events = emulator.apply_action(game_state, "call", 0)
print(events[0]["round_state"])
print(events[0]["round_state"]["community_card"])
print(events[0]["round_state"]["seats"])
print("\n")

{'round_count': 0, 'small_blind_amount': 5, 'street': 0, 'next_player': None, 'table': <pypokerengine.engine.table.Table object at 0x00000240ADDA10C0>}


{'street': 'preflop', 'pot': {'main': {'amount': 15}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_player': 1, 'small_blind_pos': 1, 'big_blind_pos': 0, 'round_count': 1, 'small_blind_amount': 5, 'seats': [{'name': 'Player 1', 'uuid': 'p1_uuid', 'stack': 990, 'state': 'participating'}, {'name': 'Player 2', 'uuid': 'p2_uuid', 'stack': 995, 'state': 'participating'}], 'action_histories': {'preflop': [{'action': 'SMALLBLIND', 'amount': 5, 'add_amount': 5, 'uuid': 'p2_uuid'}, {'action': 'BIGBLIND', 'amount': 10, 'add_amount': 5, 'uuid': 'p1_uuid'}]}}
[]
[{'name': 'Player 1', 'uuid': 'p1_uuid', 'stack': 990, 'state': 'participating'}, {'name': 'Player 2', 'uuid': 'p2_uuid', 'stack': 995, 'state': 'participating'}]


{'street': 'preflop', 'pot': {'main': {'amount': 30}, 'side': []}, 'community_card': [], 'dealer_btn': 0, 'next_